<h1><center>Auxiliar 1: Apoyo para la Competencia </center></h1>

<center><strong>CC6205: Procesamiento de Lenguaje Natural - Otoño 2022</strong></center>

## 📚 Objetivos del Notebook 📚

El objetivo principal de esta clase es presentarles otras técnicas para que afronten la competencía. 

Utilizaremos los siguientes elementos, que fueron descritos en las cátedras:

- Tokenización.
- Word2vec.
- RandomForest.

Las librerías que vamos a utilizar son las siguientes:

- Pandas
- Numpy
- Scikit-Learn
- Gensim

In [1]:
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('opinion_lexicon')
nltk.download('vader_lexicon')
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.sentiment.util import  mark_negation
from nltk.corpus import opinion_lexicon
from collections import Counter

from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

#Word2vec
import gensim
import gensim.downloader

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\imeza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\imeza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\imeza\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\imeza\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


La tarea que buscaremos resolver en este notebook es la clasificación de sentimientos dado un corpus. Para esto utilizaremos twitters enfocados en videojuegos e intentaremos clasificar texto no visualizado por un modelo. **Notar** que para efectos prácticos de este notebook se ha usado una muestra de la totalidad de datos, esto producto del alto costo computacional que exige la tarea de muchooooos documentos.

Comenzamos cargando los diferentes dataset que utilizaremos:

In [2]:
# Cargamos y le damos nuevos nombres a las columnas
train_set = pd.read_csv('./data/twitter_training.csv')
test_set  = pd.read_csv('./data/twitter_validation.csv')

train_set.columns = ['id_company','company', 'sentiment', 'corpus']
test_set.columns = ['id_company','company', 'sentiment', 'corpus']

# Condicionamos para que la tarea no tarde tanto
cond1 = (train_set['sentiment'] != "Neutral").values & \
       (train_set['sentiment'] != "Irrelevant").values & \
       (train_set['corpus'] != "<unk>").values

cond2 = (test_set['sentiment'] != "Neutral").values & \
       (test_set['sentiment'] != "Irrelevant").values & \
       (test_set['corpus'] != "<unk>").values

# Se muestrean aleatoriamente 500 ejemplos para cada clase
train_set = train_set[cond1].dropna()
df1 = train_set[train_set.sentiment == 'Positive'].sample(500, replace=True)
df2 = train_set[train_set.sentiment == 'Negative'].sample(500, replace=True)
train_set = pd.concat([df1, df2]).reset_index(drop=True)

test_set = test_set[cond2].dropna()
df1 = test_set[test_set.sentiment == 'Positive'].sample(500, replace=True)
df2 = test_set[test_set.sentiment == 'Negative'].sample(500, replace=True)
test_set = pd.concat([df1, df2]).reset_index(drop=True)

Observamos que tiene cada uno de los datasets:

In [3]:
train_set.head()

,id_company,company,sentiment,corpus
0,6367,FIFA,Positive,number 10] FIFA 19 Youth Player.. gamersdecide...
1,7351,LeagueOfLegends,Positive,Teamwork at work most... see.tv/clips/20350038...
2,974,AssassinsCreed,Positive,Soooo gassed .
3,4583,Google,Positive,Thank them guys!!!!!
4,1927,CallOfDutyBlackopsColdWar,Positive,AND DON ’ t yet be FUCKEN LATE


In [4]:
test_set.head()

,id_company,company,sentiment,corpus
0,2547,Borderlands,Positive,I love u guys
1,2450,Borderlands,Positive,The next one is Borderlands: The Handsome Coll...
2,6353,FIFA,Positive,This was easily the best Fifa. First time with...
3,3469,Facebook,Positive,I am very interested in this
4,2450,Borderlands,Positive,The next one is Borderlands: The Handsome Coll...


## ¿Existe una forma de obtener caracteristicas desde un corpus?

Un ejercicio que se suele hacer es utilizar lexicones para obtener caracteristicas desde un corpus. Los pro de esta tarea es que usamos un conjunto de datos etiquetado manualmente por personas, que logran "identificar" el sentimiento de un conjunto de palabras. El punto negativo, es que el etiquetado tiene una gran subjetividad y se ve reducida a un conjunto de palabras.

Para este ejercicio, utilizaremos un lexicon de sentimientos de [Bing Liu](https://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html) que tiene palabras etiquetadas según el sentimiento positivo y negativo. El contenido de este lexicon distribuye de la siguiente forma:

In [5]:
print(f"Cantidad de palabras positivas: {len(opinion_lexicon.positive())}")
print(f"Cantidad de palabras positivas: {len(opinion_lexicon.negative())}")

Cantidad de palabras positivas: 2006
Cantidad de palabras positivas: 4783


Donde podemos encontrar las siguientes palabras:

In [6]:
list(opinion_lexicon.positive()+opinion_lexicon.negative())

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation',
 'accolade',
 'accolades',
 'accommodative',
 'accomodative',
 'accomplish',
 'accomplished',
 'accomplishment',
 'accomplishments',
 'accurate',
 'accurately',
 'achievable',
 'achievement',
 'achievements',
 'achievible',
 'acumen',
 'adaptable',
 'adaptive',
 'adequate',
 'adjustable',
 'admirable',
 'admirably',
 'admiration',
 'admire',
 'admirer',
 'admiring',
 'admiringly',
 'adorable',
 'adore',
 'adored',
 'adorer',
 'adoring',
 'adoringly',
 'adroit',
 'adroitly',
 'adulate',
 'adulation',
 'adulatory',
 'advanced',
 'advantage',
 'advantageous',
 'advantageously',
 'advantages',
 'adventuresome',
 'adventurous',
 'advocate',
 'advocated',
 'advocates',
 'affability',
 'affable',
 'affably',
 'affectation',
 'affection',
 'affectionate',
 'affinity',
 'affirm',
 'affirmation',
 'affirmative',
 'affluence',
 'affluent',
 'afford',
 'affordable',
 'af

**Todo muy bonito, ¿como puedo usar eso?...** 🤔

Una forma es crear una función que nos permita contar la cantidad de palabras positivas y negativas que existen por documento. Con esto buscaremos crear una función que nos permita mapear un conjunto de palabras a un vector de largo dos por documento (puede sonar poco pero el aporte puede ser relevante para un clasificador).

In [7]:
class LiuFeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.pos_set = set(opinion_lexicon.positive())
        self.neg_set = set(opinion_lexicon.negative())

    def __liu_score(self,sentence):
        tokenized_sent = self.tokenizer.tokenize(sentence)
        pos_words, neg_words = 0, 0
        for word in tokenized_sent:
            if word in self.pos_set:
                pos_words += 1
            elif word in self.neg_set:
                neg_words += 1
        return [pos_words,neg_words]

    def transform(self, X, y=None):
        values = []
        for tweet in X:
            values.append(self.__liu_score(tweet))

        return(np.array(values))

    def fit(self, X, y=None):
        return self

In [8]:
tokenizador= TweetTokenizer(preserve_case=False, reduce_len=True)

sentimentAnalisis2 = LiuFeatureExtractor(tokenizer = tokenizador)
sentimentAnalisis2.fit_transform(X = train_set['corpus'])

array([[1, 0],
       [1, 0],
       [0, 0],
       ...,
       [0, 1],
       [0, 1],
       [1, 0]])

## Word2Vec 📚📚

<center><img src="https://multithreaded.stitchfix.com/assets/posts/2016-05-27-lda2vec/anim00.gif" width="400"></center>

Word2vec es una tecnica mas sofisticada que les permitira obtener representaciones vectoriales mas "ricas". Esto producto que Word2vec entrena una red neuronal superficial para obtener a traves de los pesos, representaciones que pueden absorber parte de la semantica representada en un documento.

Mas información se dará en la proxima auxiliar......

### Ejemplo Practico con Word2Vec 🛐

Con tal de ayudarlos en su competencia, las siguientes celdas exponen una forma resumida de como utilizar Word2Vec. Para esto pueden tomar 2 caminos que es el **entrenamiento** de un modelo propio, o utilizar modelos **pre-entrenados** que se encuentran en la red. Para efectos de la competencia se les recomienda el segundo, ya que pueden existir modelos pre-entrenados en corpus gigantescos que les permitan absorber mejor información desde el corpus.

**Cargar modelos Pre-entrenados:** A continuación, cargaremos un modelo pre-entrenado llamado glove para la clasificación de los sentimientos expuestos en la primera sección. La carga es simple y se hace especificando a gensim el modelo pre-entrenado que quieran cargar de su librería.

In [9]:
pretrained_model = gensim.downloader.load('glove-twitter-100')

Como pueden ver la forma anterior solo es capaz de cargar modelos predefinidos, para cargar modelos que poseen en un archivo `.bin` deberán cargarlo de la siguiente manera:

```python
model_pretrained = gensim.models.KeyedVectors.load_word2vec_format("word2vec_twitter_tokens.bin", 
                                                                   binary=True, 
                                                                   unicode_errors='ignore')
```

**Word2Vec en acción:** Una vez cargado el modelo, podemos ver el vocabulario de este con el siguiente comando:

In [10]:
pretrained_model.vocab

{'<user>': <gensim.models.keyedvectors.Vocab at 0x209ae064580>,
 '.': <gensim.models.keyedvectors.Vocab at 0x209ae0641f0>,
 ':': <gensim.models.keyedvectors.Vocab at 0x209ae064d00>,
 'rt': <gensim.models.keyedvectors.Vocab at 0x209ae064880>,
 ',': <gensim.models.keyedvectors.Vocab at 0x209ae064940>,
 '<repeat>': <gensim.models.keyedvectors.Vocab at 0x209ae064a00>,
 '<hashtag>': <gensim.models.keyedvectors.Vocab at 0x209ae064850>,
 '<number>': <gensim.models.keyedvectors.Vocab at 0x209ae0647c0>,
 '<url>': <gensim.models.keyedvectors.Vocab at 0x209ae0646d0>,
 '!': <gensim.models.keyedvectors.Vocab at 0x209ae064cd0>,
 'i': <gensim.models.keyedvectors.Vocab at 0x209ae064a30>,
 'a': <gensim.models.keyedvectors.Vocab at 0x209ae064490>,
 '"': <gensim.models.keyedvectors.Vocab at 0x209ae064340>,
 'the': <gensim.models.keyedvectors.Vocab at 0x209ae064c40>,
 '?': <gensim.models.keyedvectors.Vocab at 0x209ae064c70>,
 'you': <gensim.models.keyedvectors.Vocab at 0x209ae0646a0>,
 'to': <gensim.model

De la celda anterior, podemos var cada una de las palabras utilizadas en el entrenamiento y el vector asociado a cada palabra. Otras operaciones interesantes con el modelo es la visualización de las palabras que tienen mayor similitud (según la similitud de coseno), la que viene dada por:

In [11]:
pretrained_model.most_similar("president")

[('obama', 0.8386131525039673),
 ('barack', 0.8213216662406921),
 ('clinton', 0.761232852935791),
 ('minister', 0.7525885105133057),
 ('prez', 0.7500050067901611),
 ('says', 0.7496355175971985),
 ('romney', 0.7477604150772095),
 ('government', 0.7370582818984985),
 ('elected', 0.7291470766067505),
 ('potus', 0.724652886390686)]

**y.. ¿Comó clasificamos?....** ❓

Para la clasificación, realizaremos algo muy similar a la obtención de caracteristicas según un lexicon. En esta misma linea, utilizaremos el modelo para obtener los vectores que representan a cada una de las palabras que tenemos en un documento. Una vez obtenidas las palabras de un documento, promediaremos los vectores para generar una representación vectorial del documento (en esta etapa no es necesario que utilicen solo la media, pueden usar mediana, std, lo que quieran).

In [12]:
class Doc2Vector(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.model = pretrained_model
        self.vocab = list(pretrained_model.vocab.keys())
    
    def to_vector(self, sentence):
        
        tokenized_sent = self.tokenizer.tokenize(sentence)
        doc_vector = np.empty(self.model.vector_size, dtype=float)
        
        for word in tokenized_sent:
            if word in self.vocab:
                word_vector = self.model[word]
                doc_vector  = np.vstack((doc_vector, word_vector))
         
        try:
            doc_vector = doc_vector[1:].mean(axis=0)
            return list(doc_vector)
        except:
            return None
        
    def transform(self, X, y=None):
        values = []
        for tweet in X:
            vector = self.to_vector(tweet)
            values.append(vector)
                
        return(np.squeeze(np.array(values)))

    def fit(self, X, y=None):
        return self

Probamos el extractor creado:

In [13]:
tokenizador= TweetTokenizer(preserve_case=False, reduce_len=True)

doc2vec = Doc2Vector(tokenizer=tokenizador)
embeddings = doc2vec.transform(X = train_set['corpus'])

<ipython-input-12-dc65bc65cc52>:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return(np.squeeze(np.array(values)))


Veamos lo que generamos:

In [14]:
documentos_vectores = pd.DataFrame(zip(train_set['corpus'], embeddings, train_set['sentiment']), columns=['documento', 'embedding', 'label'])
documentos_vectores = documentos_vectores.dropna()
documentos_vectores.head(10)

,documento,embedding,label
0,number 10] FIFA 19 Youth Player.. gamersdecide...,"[0.2670258767902851, -0.3495679940097034, 0.12...",Positive
1,Teamwork at work most... see.tv/clips/20350038...,"[-0.18857750296592712, -0.03230975056067109, 0...",Positive
2,Soooo gassed .,"[-4.6665469805399575e-05, -0.03462066625555356...",Positive
3,Thank them guys!!!!!,"[0.035782103116313614, 0.39485832800467807, -0...",Positive
4,AND DON ’ t yet be FUCKEN LATE,"[-0.08975087525323033, 0.23694325052201748, -0...",Positive
5,I started playing Overwatch (on Switch) right ...,"[0.19826787104830146, 0.06786094610824396, 0.0...",Positive
6,From Nice,"[0.27478499338030815, -0.15036050509661436, -0...",Positive
7,enjoying league of legends atm pic.twitter.com...,"[-0.09384039938449859, -0.10986479893326759, 0...",Positive
8,So how does not everyone feel about,"[0.05222557046051536, 0.17267985748393194, 0.3...",Positive
9,Oh her goodness.,"[0.213899752125144, -0.08656825125217438, 0.16...",Positive


In [15]:
documentos_vectores = pd.concat([documentos_vectores[['documento','label']], 
                                 documentos_vectores['embedding'].apply(pd.Series)], 
                                axis = 1)

**Entrenamiento 🥋**

Finalmente, entrenemos un modelo y veamos que tan bien le va con un `RandomForestClassifier()` 🌳:

In [16]:
# Comenzamos separando el conjunto de datos
X = documentos_vectores.drop(columns=['documento','label'])
y = documentos_vectores['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [17]:
# Usamos un randomforest por default
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

RandomForestClassifier()

In [18]:
# Creamos un dataframe para exponer los resultados
labels_predicted = clf.predict(X_test)

casos_predichos = documentos_vectores.iloc[X_test.index][['documento', 'label']].reset_index(drop=True)
casos_predichos = pd.concat([casos_predichos, pd.DataFrame(labels_predicted, columns=['predicted_label'])], axis=1)
casos_predichos.head()

,documento,label,predicted_label
0,to,Positive,Positive
1,What is your ban procedure @ UbisoftSupport!? ...,Negative,Negative
2,We didn't deserve Battlefield 1.,Positive,Negative
3,Notes When first the PS4 player and Xbox One c...,Positive,Positive
4,sometimes i feel like such stupid old grandma ...,Negative,Negative


In [19]:
# su accuracy para calcular que tal le va al chico de las poesias
acc = (casos_predichos['label'] == casos_predichos['predicted_label']).sum() / casos_predichos['label'].shape[0]

print(f"accuracy: {acc}")

accuracy: 0.793939393939394


Del resultado, podemos observar que a pesar de que no hicimos mucho nuestro modelo y el extractor de caracteristicas Doc2vec logran clasificar bien al conjunto de prueba.